In [1]:
import sys
sys.path.append("../../FinNLP")

### SEC

In [2]:
from finnlp.data_sources.company_announcement.sec import SEC_Announcement

In [3]:
start_date = "2020-01-01"
end_date = "2020-06-01"
stock = "AAPL"
config = {
    "use_proxy": "us_free",
    "max_retry": 5,
    "proxy_pages": 3,
}


In [4]:
downloader = SEC_Announcement(config)
downloader.download_date_range_stock(start_date, end_date, stock = stock)

Checking ips: 100%|██████████| 45/45 [01:42<00:00,  2.28s/it]


Get proxy ips: 45.
Usable proxy ips: 44.


In [7]:
df = downloader.dataframe
# df = df.drop_duplicates()
df.head(2)

,_id,ciks,period_ending,root_form,file_num,display_names,xsl,sequence,file_date,biz_states,sics,form,adsh,film_num,biz_locations,file_type,file_description,inc_states,ite,content
0,0000320193-20-000056:wf-form4_158932261319105.xml,"[0001631982, 0000320193]",2020-05-08,4,[],"[KONDO CHRIS (CIK 0001631982), Apple Inc. (A...",xslF345X03,1,2020-05-12,[],[3571],4,0000320193-20-000056,[],"[, ]",4,FORM 4,"[, CA, ]",[],SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
1,0000320193-20-000054:wf-form4_158829658358801.xml,"[0001051401, 0000320193]",2020-04-28,4,[001-36743],"[JUNG ANDREA (CIK 0001051401), Apple Inc. (A...",xslF345X03,1,2020-04-30,[CA],[3571],4,0000320193-20-000054,[20838087],"[, Cupertino, CA]",4,FORM 4,"[, CA]",[],SEC Form 4 \n FORM 4UNITED STATES SECURITIES...


In [8]:
df.shape

(21, 20)

In [9]:
selected_columns = ["file_date", "display_names", "content"]
df[selected_columns].head(10)

,file_date,display_names,content
0,2020-05-12,"[KONDO CHRIS (CIK 0001631982), Apple Inc. (A...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
1,2020-04-30,"[JUNG ANDREA (CIK 0001051401), Apple Inc. (A...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
2,2020-04-17,"[O'BRIEN DEIRDRE (CIK 0001767094), Apple Inc....",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
3,2020-04-17,"[KONDO CHRIS (CIK 0001631982), Apple Inc. (A...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
4,2020-04-09,"[Maestri Luca (CIK 0001513362), Apple Inc. (...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
5,2020-04-03,"[WILLIAMS JEFFREY E (CIK 0001496686), Apple I...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
6,2020-04-03,"[Maestri Luca (CIK 0001513362), Apple Inc. (...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
7,2020-02-28,"[WAGNER SUSAN (CIK 0001059235), Apple Inc. (...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
8,2020-02-28,"[LEVINSON ARTHUR D (CIK 0001214128), Apple In...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...
9,2020-02-28,"[JUNG ANDREA (CIK 0001051401), Apple Inc. (A...",SEC Form 4 \n FORM 4UNITED STATES SECURITIES...


### Juchao

In [10]:
from finnlp.data_sources.company_announcement.juchao import Juchao_Announcement

In [11]:
start_date = "2020-01-01"
end_date = "2020-06-01"
stock = "000001"
config = {
    "use_proxy": "china_free",
    "max_retry": 5,
    "proxy_pages": 3,
}


In [14]:
downloader = Juchao_Announcement(config)
downloader.download_date_range_stock(start_date, end_date, stock = stock, get_content = True, delate_pdf = True)

Checking ips: 100%|██████████| 45/45 [00:48<00:00,  1.09s/it]


获取到的代理ip数量: 45 。Get proxy ips: 45.
能用的代理数量： 6。Usable proxy ips: 6.


Getting the text data...:   0%|          | 0/42 [00:00<?, ?it/s]

In [15]:
df = downloader.dataframe
df.head(2)

,id,secCode,secName,orgId,announcementId,announcementTitle,announcementTime,adjunctUrl,adjunctSize,adjunctType,...,important,batchNum,announcementContent,orgName,tileSecName,shortTitle,announcementTypeName,secNameList,PDF_path,Content
0,None,000001,平安银行,gssz0000001,1207862647,关于2020年第一期小型微型企业贷款专项金融债券发行完毕的公告,2020-05-27,finalpage/2020-05-27/1207862647.PDF,148,PDF,...,None,None,,None,平安银行,关于2020年第一期小型微型企业贷款专项金融债券发行完毕的公告,None,None,removed,证券代码： 000001 证券简称：平安银行 ...
1,None,000001,平安银行,gssz0000001,1207843688,2019年年度权益分派实施公告,2020-05-22,finalpage/2020-05-22/1207843688.PDF,214,PDF,...,None,None,,None,平安银行,2019年年度权益分派实施公告,None,None,removed,1 证券代码： 000001 证券简称：平安银行 ...


In [16]:
df.shape

(42, 25)

In [17]:
selected_columns = ["announcementTime", "shortTitle","Content"]
df[selected_columns].head(10)

,announcementTime,shortTitle,Content
0,2020-05-27,关于2020年第一期小型微型企业贷款专项金融债券发行完毕的公告,证券代码： 000001 证券简称：平安银行 ...
1,2020-05-22,2019年年度权益分派实施公告,1 证券代码： 000001 证券简称：平安银行 ...
2,2020-05-20,关于获准发行小微企业贷款专项金融债券的公告,证券代码： 000001 证券简称：平安银行 ...
3,2020-05-16,监事会决议公告,1 证券代码： 000001 证券简称： 平安银行 ...
4,2020-05-15,2019年年度股东大会决议公告,1 证券代码： 000001 证券简称：平安银行 ...
5,2020-05-15,2019年年度股东大会的法律意见书,北京总部 电话 : (86 -10) 8519 -1300 传真 : (86 -10...
6,2020-04-30,中信证券股份有限公司、平安证券股份有限公司关于公司关联交易有关事项的核查意见,1 中信证券股份有限公司 、平安证券股份有限 公司 关于平安银行股份有限公司 关联交易 有...
7,2020-04-30,独立董事独立意见,1 平安银行股份有限公司独立董事独立意见 根据《关于在上市公司建立独立董事制度的指导...
8,2020-04-30,关联交易公告,1 证券代码： 000001 证券简称：平安银行 ...
9,2020-04-21,2020年第一季度报告全文,证券代码： 000001 证券简称：平安银行 ...
